In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import transformers

import sys
sys.path.append('..')
from src.attentionmlp import AttentionMLP


In [2]:
# load tokenized data
path_train ='ECHR_Dataset_Tokenized/legal-bert-base-uncased/df_dev_tokenized.pkl'
path_dev ='ECHR_Dataset_Tokenized/legal-bert-base-uncased/df_dev_tokenized.pkl'
path_test ='ECHR_Dataset_Tokenized/legal-bert-base-uncased/df_test_tokenized.pkl'

df_train = pd.read_pickle(path_train)
df_dev = pd.read_pickle(path_dev)
df_test = pd.read_pickle(path_test)

In [3]:
df_train.head()

,ITEMID,LANGUAGEISOCODE,RESPONDENT,BRANCH,DATE,DOCNAME,IMPORTANCE,CONCLUSION,JUDGES,text,VIOLATED_ARTICLES,VIOLATED_PARAGRAPHS,VIOLATED_BULLETPOINTS,NON_VIOLATED_ARTICLES,NON_VIOLATED_PARAGRAPHS,NON_VIOLATED_BULLETPOINTS,label,input_ids,attention_mask
0,001-100036,ENG,LTU,CHAMBER,2010,CASE OF BALČIŪNAS v. LITHUANIA,4,Violation of Art. 5-3;No violation of Art. 6-1...,Alvina Gyulumyan;Corneliu Bîrsan;Egbert Myjer;...,[7. The applicant was born in 1977 and lives i...,[5],[5-3],[],[6],"[6-1, 6-3]",[6-3-d],1,"[[[tensor(101), tensor(204), tensor(117), tens...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
1,001-100094,ENG,ARM,ADMISSIBILITY,2010,PAPYAN AND DAVTYAN v. ARMENIA,4,Inadmissible,Alvina Gyulumyan;Elisabet Fura;Ineta Ziemele;J...,"[1. The applicants, Mr Hayk Papyan, Mr Samvel ...",[],[],[],[],[],[],0,"[[[tensor(101), tensor(198), tensor(117), tens...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
2,001-100139,ENG,HUN,COMMITTEE,2010,CASE OF GYARFAS AND HUNAUDIT KFT. v. HUNGARY,4,Violation of Art. 6-1,András Sajó;Kristina Pardalos,[4. The applicant was born in 1956 and lives i...,[6],[6-1],[],[],[],[],1,"[[[tensor(101), tensor(201), tensor(117), tens...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
3,001-100141,ENG,TUR,CHAMBER,2010,CASE OF ABDOLKHANI AND KARIMNIA v. TURKEY (no. 2),4,Violation of Art. 3,Françoise Tulkens;Guido Raimondi;Ireneu Cabral...,[4. The applicants were born in 1973 and 1978 ...,[3],[],[],[],[],[],1,"[[[tensor(101), tensor(201), tensor(117), tens...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
4,001-100197,ENG,LTU,CHAMBER,2010,CASE OF GINEITIENE v. LITHUANIA,3,Remainder inadmissible;No violation of Art. 14...,András Sajó;Françoise Tulkens;Guido Raimondi;K...,[5. The applicant was born in 1968 and lives i...,[],[],[],"[14, 8]",[],[],0,"[[[tensor(101), tensor(202), tensor(117), tens...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."


In [4]:
documents = df_train[['input_ids', 'attention_mask', 'label']]

In [5]:
# convert the series into a list
input_ids = documents.input_ids.tolist()
attention_mask = documents.attention_mask.tolist()
labels = documents.label.tolist()

In [6]:
input_ids = [torch.stack(i) for i in input_ids]
attention_mask = [torch.stack(i) for i in attention_mask]


In [7]:
input_ids =[torch.squeeze(i, dim=1) for i in input_ids]
attention_mask =[torch.squeeze(i, dim=1) for i in attention_mask]

In [8]:
lengths =[i.size(0) for i in input_ids]

In [9]:
def collate_fn(data):
    """
       data: is a list of tuples with (input_ids, attention mask, label, length)
    """
    input_ids = [i[0] for i in data]
    attention_mask = [i[1] for i in data]
    labels = [i[2] for i in data]
    lengths = [i[3] for i in data]


    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)
    max_length = torch.max(lengths)
    # pad the input_ids and attention_mask so that they have the same length [max_length, 512]
    for i in range(len(input_ids)):
        pad = torch.zeros((max_length - lengths[i],512), dtype=torch.long)
        input_ids[i] = torch.cat((input_ids[i], pad), dim=0, )
        attention_mask[i] = torch.cat((attention_mask[i], pad), dim=0)

    return torch.stack(input_ids), torch.stack(attention_mask), labels, lengths

In [10]:
class ECHRDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx], self.input_ids[idx].size(0) # last one is the length of the input_ids, used for padding

In [11]:
# create a dataset
dataset = ECHRDataset(input_ids, attention_mask, labels)

In [12]:
# create a dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

In [13]:
x=next(iter(dataloader))

len(x)

4

In [14]:
x[0].shape

torch.Size([1, 4, 512])

In [15]:
# try to run the model
from transformers import BertModel
bert = BertModel.from_pretrained('nlpaueb/legal-bert-base-uncased')

In [16]:
def make_mask(data, lengths, batch_first=True):
    if batch_first:
        max_length = data.size(1)
        batch_size = data.size(0)
    else:
        max_length = data.size(0)
        batch_size = data.size(1)
    mask = torch.zeros((max_length, batch_size), dtype=torch.bool)

    for i, l in enumerate(lengths):
        mask[i, :l] = 1.

    return mask


class Hierbert(nn.Module):
    def __init__(self, bert, hidden_sizes ):
        super(Hierbert, self).__init__()
        self.bert = bert
        self.attention_mlp = AttentionMLP(768, hidden_sizes)

    def forward(self, input_ids, attention_masks, lengths, bert_require_grad=True):

        max_l = input_ids.size(1)# inputs are already padded
        bert_output = []

        if bert_require_grad:
            self.bert.train()
        else:
            self.bert.eval()

        for i in range(max_l):
            bert_output.append(
                self.bert(input_ids[:,i], attention_masks[:,i]).pooler_output
            )

        bert_output = torch.stack(bert_output)

        print(bert_output.shape)
        sentence_mask = make_mask(input_ids, lengths).to('cuda')

        return self.attention_mlp(bert_output.permute(1,0,2), sentence_mask.T)

In [17]:
len(x)
x

(tensor([[[101, 207, 272,  ...,   0,   0,   0],
          [101, 213, 145,  ...,   0,   0,   0],
          [101, 222, 203,  ..., 117, 102,   0],
          [101, 242, 248,  ...,   0,   0,   0]]]),
 tensor([[[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 0],
          [1, 1, 1,  ..., 0, 0, 0]]]),
 tensor([0]),
 tensor([4]))

In [18]:
from accelerate import Accelerator
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch

device = 'cuda'
model = Hierbert(bert=bert, hidden_sizes=[768,2])

optimizer = Adam(model.parameters(), lr=0.001)
accelerator = Accelerator()

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
loss_function = torch.nn.BCELoss()

model, optimizer, training_dataloader, scheduler = accelerator.prepare(
     model, optimizer, dataloader, scheduler
)

for batch in training_dataloader:
    optimizer.zero_grad()
    inputs, att, targets, l = batch
    inputs = inputs
    targets = targets
    outputs = model(inputs, att, l)
    loss = loss_function(outputs, targets.float())
    accelerator.backward(loss)
    optimizer.step()
    scheduler.step()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


torch.Size([18, 1, 768])
torch.Size([10, 1, 768])
torch.Size([8, 1, 768])
torch.Size([10, 1, 768])
torch.Size([6, 1, 768])


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 18.12 MiB is free. Process 2253390 has 1.39 GiB memory in use. Process 2253391 has 822.00 MiB memory in use. Including non-PyTorch memory, this process has 13.68 GiB memory in use. Of the allocated memory 12.98 GiB is allocated by PyTorch, and 15.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)